In [1]:
# See covid_virtual_env.sh for environment setup
import boto3, pandas as pd, pickle

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_rows', 500)

# Download Data

Set the client for boto3 (amazons toolkit) to read s3

In [2]:
client = boto3.client('s3')

In [3]:
# Download the census metadata
obj = client.get_object(Bucket='covid19-lake', Key='safegraph-open-census-data/csv/metadata/cbg_fips_codes/cbg_fips_codes.csv')
data_fips = pd.read_csv(obj['Body'])

obj = client.get_object(Bucket='covid19-lake', Key='safegraph-open-census-data/csv/metadata/cbg_geographic_data/cbg_geographic_data.csv')
data_geo = pd.read_csv(obj['Body'])

In [4]:
keys = [
    'safegraph-open-census-data/csv/data/cbg_b01.csv', #Age/Gender
    'safegraph-open-census-data/csv/data/cbg_b02.csv', #Race
    'safegraph-open-census-data/csv/data/cbg_b15.csv', #Education
    'safegraph-open-census-data/csv/data/cbg_b19.csv', #Income
    'safegraph-open-census-data/csv/data/cbg_b27.csv', #Insurance
    'safegraph-open-census-data/csv/data/cbg_c17.csv'  #Poverty
]
data_list = []
for key in keys:
    obj = client.get_object(Bucket='covid19-lake', Key=key)
    data = pd.read_csv(obj['Body'])
    data_list.append(data)
    print('Added: ', key)

Added:  safegraph-open-census-data/csv/data/cbg_b01.csv
Added:  safegraph-open-census-data/csv/data/cbg_b02.csv
Added:  safegraph-open-census-data/csv/data/cbg_b15.csv
Added:  safegraph-open-census-data/csv/data/cbg_b19.csv
Added:  safegraph-open-census-data/csv/data/cbg_b27.csv
Added:  safegraph-open-census-data/csv/data/cbg_c17.csv


In [5]:
new_data_list = []
for data in data_list:
    data = data.sort_values(by='census_block_group', ascending=True)
    new_data_list.append(data)

In [62]:
x = 25000
new_data_list[0][x:x+5]

,census_block_group,B01001e1,B01001m1,B01001e2,B01001m2,B01001e3,B01001m3,B01001e4,B01001m4,B01001e5,...,B01002He3,B01002Hm3,B01002Ie1,B01002Im1,B01002Ie2,B01002Im2,B01002Ie3,B01002Im3,B01003e1,B01003m1
25000,60670089092,965,252,453,158,50,47,0,12,63,...,48.4,5.7,27.1,13.3,22.8,16.2,32.1,19.0,965,252
25001,60670089101,1561,326,853,205,46,44,51,51,61,...,41.0,10.0,38.2,10.0,44.1,18.8,28.9,19.3,1561,326
25002,60670089102,1944,409,931,306,48,53,69,66,52,...,48.3,7.8,28.7,3.5,29.3,14.0,28.3,4.1,1944,409
25003,60670089111,860,120,445,70,72,45,22,19,25,...,49.1,13.7,23.0,5.7,16.8,10.6,26.6,5.8,860,120
25004,60670089112,1452,267,710,128,90,71,56,31,44,...,28.9,6.5,19.0,7.0,16.8,11.0,22.8,8.6,1452,267


In [59]:
new_data_list[0].shape

(220333, 161)

In [64]:
census_data = pd.concat(new_data_list, axis=1)

In [65]:
census_data.shape

(220333, 1414)

In [66]:
with open('safegraph_census_raw.p', 'wb') as f:
    pickle.dump(census_data,f)

In [67]:
census_data.head()

,census_block_group,B01001e1,B01001m1,B01001e2,B01001m2,B01001e3,B01001m3,B01001e4,B01001m4,B01001e5,...,B17021e31,B17021m31,B17021e32,B17021m32,B17021e33,B17021m33,B17021e34,B17021m34,B17021e35,B17021m35
0,10010201001,745,226,356,121,33,32,46,39,4,...,88,42,66,37,56,35,10,12,22,27
1,10010201002,1265,232,639,139,3,6,6,9,53,...,64,36,58,32,50,30,8,11,6,9
2,10010202001,960,261,534,176,0,11,82,71,90,...,72,38,62,33,62,33,0,11,10,14
3,10010202002,1236,227,634,136,28,21,47,39,33,...,145,51,138,50,132,51,6,9,7,10
4,10010203001,2364,378,1125,229,20,24,166,92,92,...,268,99,249,90,213,88,36,35,19,23


In [68]:
#with open('safegraph_census_raw.p', 'rb') as f:
#    census_data = pickle.load(f)

In [69]:
# Download all the census data
#lake = 'covid19-lake'
#folder = 'safegraph-open-census-data/csv/data'
#objects = client.list_objects(Bucket=lake, Prefix=folder)['Contents']
#data_list = []

#for object in objects:
#    tmp = object['Key']
#    obj = client.get_object(Bucket=lake, Key=tmp)
#    data = pd.read_csv(obj['Body'])
#    data_list.append(data)
#    print("Added: ", key)

# Selecting Census Data Fields

In [70]:
obj = client.get_object(Bucket='covid19-lake', Key='safegraph-open-census-data/csv/metadata/cbg_field_descriptions/cbg_field_descriptions.csv')
data_fields = pd.read_csv(obj['Body'])

In [71]:
# Print out all data fields avaialble for searching for relavant information
data_fields.to_csv('cencus_all_data_fields.csv')

In [72]:
# Add table ID needed, consistent with file names
data_fields['table'] = data_fields['table_id'].str[0:3]

In [73]:
# After inspection, filter to a small list of tables
tables = ['B01', 'B02', 'B03', 'B15', 'B19', 'B20', 'B27', 'C17']
data_fields = data_fields[data_fields['table'].isin(tables)]

In [74]:
# Drop the margin of error columns
data_fields = data_fields[~data_fields['field_full_name'].str.contains("margin of error", case=False)]

In [75]:
# Reduced down the number of columns significantly
data_fields.shape

(562, 11)

In [76]:
# After manual inspection, these are the selected columns with their original and new more readable name

fields = [
    ['B01001e1','pop_total'], #SEX BY AGE: Total: Total population -- (Estimate)

    ['B01001e2','m_total'], #SEX BY AGE: Male: Total population -- (Estimate)
    ['B01001e3','m_0_5'], #SEX BY AGE: Male: Under 5 years: Total population -- (Estimate)
    ['B01001e4','m_5_9'], #SEX BY AGE: Male: 5 to 9 years: Total population -- (Estimate)
    ['B01001e5','m_10_14'], #SEX BY AGE: Male: 10 to 14 years: Total population -- (Estimate)
    ['B01001e6','m_15_17'], #SEX BY AGE: Male: 15 to 17 years: Total population -- (Estimate)
    ['B01001e7','m_18_19'], #SEX BY AGE: Male: 18 and 19 years: Total population -- (Estimate)
    ['B01001e8','m_20_20'], #SEX BY AGE: Male: 20 years: Total population -- (Estimate)
    ['B01001e9','m_21_21'], #SEX BY AGE: Male: 21 years: Total population -- (Estimate)
    ['B01001e10','m_22_24'], #SEX BY AGE: Male: 22 to 24 years: Total population -- (Estimate)
    ['B01001e11','m_25_29'], #SEX BY AGE: Male: 25 to 29 years: Total population -- (Estimate)
    ['B01001e12','m_30_34'], #SEX BY AGE: Male: 30 to 34 years: Total population -- (Estimate)
    ['B01001e13','m_35_39'], #SEX BY AGE: Male: 35 to 39 years: Total population -- (Estimate)
    ['B01001e14','m_40_44'], #SEX BY AGE: Male: 40 to 44 years: Total population -- (Estimate)
    ['B01001e15','m_45_49'], #SEX BY AGE: Male: 45 to 49 years: Total population -- (Estimate)
    ['B01001e16','m_50_54'], #SEX BY AGE: Male: 50 to 54 years: Total population -- (Estimate)
    ['B01001e17','m_55_59'], #SEX BY AGE: Male: 55 to 59 years: Total population -- (Estimate)
    ['B01001e18','m_60_61'], #SEX BY AGE: Male: 60 and 61 years: Total population -- (Estimate)
    ['B01001e19','m_62_64'], #SEX BY AGE: Male: 62 to 64 years: Total population -- (Estimate)
    ['B01001e20','m_65_66'], #SEX BY AGE: Male: 65 and 66 years: Total population -- (Estimate)
    ['B01001e21','m_67_69'], #SEX BY AGE: Male: 67 to 69 years: Total population -- (Estimate)
    ['B01001e22','m_70_74'], #SEX BY AGE: Male: 70 to 74 years: Total population -- (Estimate)
    ['B01001e23','m_75_79'], #SEX BY AGE: Male: 75 to 79 years: Total population -- (Estimate)
    ['B01001e24','m_80_84'], #SEX BY AGE: Male: 80 to 84 years: Total population -- (Estimate)
    ['B01001e25','m_85_110'], #SEX BY AGE: Male: 85 years and over: Total population -- (Estimate)

    ['B01001e26','f_total'], #SEX BY AGE: Female: Total population -- (Estimate)
    ['B01001e27','f_0_5'], #SEX BY AGE: Female: Under 5 years: Total population -- (Estimate)
    ['B01001e28','f_5_9'], #SEX BY AGE: Female: 5 to 9 years: Total population -- (Estimate)
    ['B01001e29','f_10_14'], #SEX BY AGE: Female: 10 to 14 years: Total population -- (Estimate)
    ['B01001e30','f_15_17'], #SEX BY AGE: Female: 15 to 17 years: Total population -- (Estimate)
    ['B01001e31','f_18_19'], #SEX BY AGE: Female: 18 and 19 years: Total population -- (Estimate)
    ['B01001e32','f_20_20'], #SEX BY AGE: Female: 20 years: Total population -- (Estimate)
    ['B01001e33','f_21_21'], #SEX BY AGE: Female: 21 years: Total population -- (Estimate)
    ['B01001e34','f_22_24'], #SEX BY AGE: Female: 22 to 24 years: Total population -- (Estimate)
    ['B01001e35','f_25_29'], #SEX BY AGE: Female: 25 to 29 years: Total population -- (Estimate)
    ['B01001e36','f_30_34'], #SEX BY AGE: Female: 30 to 34 years: Total population -- (Estimate)
    ['B01001e37','f_35_39'], #SEX BY AGE: Female: 35 to 39 years: Total population -- (Estimate)
    ['B01001e38','f_40_44'], #SEX BY AGE: Female: 40 to 44 years: Total population -- (Estimate)
    ['B01001e39','f_45_49'], #SEX BY AGE: Female: 45 to 49 years: Total population -- (Estimate)
    ['B01001e40','f_50_54'], #SEX BY AGE: Female: 50 to 54 years: Total population -- (Estimate)
    ['B01001e41','f_55_59'], #SEX BY AGE: Female: 55 to 59 years: Total population -- (Estimate)
    ['B01001e42','f_60_61'], #SEX BY AGE: Female: 60 and 61 years: Total population -- (Estimate)
    ['B01001e43','f_62_64'], #SEX BY AGE: Female: 62 to 64 years: Total population -- (Estimate)
    ['B01001e44','f_65_66'], #SEX BY AGE: Female: 65 and 66 years: Total population -- (Estimate)
    ['B01001e45','f_67_69'], #SEX BY AGE: Female: 67 to 69 years: Total population -- (Estimate)
    ['B01001e46','f_70_74'], #SEX BY AGE: Female: 70 to 74 years: Total population -- (Estimate)
    ['B01001e47','f_75_79'], #SEX BY AGE: Female: 75 to 79 years: Total population -- (Estimate)
    ['B01001e48','f_80_84'], #SEX BY AGE: Female: 80 to 84 years: Total population -- (Estimate)
    ['B01001e49','f_85_110'], #SEX BY AGE: Female: 85 years and over: Total population -- (Estimate)

    ['B02001e1','r_total'],  #RACE: Total: Total population -- (Estimate)
    ['B02001e2','r_white'], #RACE: White alone: Total population -- (Estimate)
    ['B02001e3','r_black'], #RACE: Black or African American alone: Total population -- (Estimate)
    ['B02001e4','r_native'], #RACE: American Indian and Alaska Native alone: Total population -- (Estimate)
    ['B02001e5','r_asian'], #RACE: Asian alone: Total population -- (Estimate)
    ['B02001e6','r_pacific'], #RACE: Native Hawaiian and Other Pacific Islander alone: Total population -- (Estimate)
    ['B02001e7','r_other'], #RACE: Some other race alone: Total population -- (Estimate)
    ['B02001e8','r_mix1'], #RACE: Two or more races: Total population -- (Estimate)
    ['B02001e9','r_mix2'], #RACE: Two or more races: Two races including Some other race: Total population -- (Estimate)
    ['B02001e10','r_mix3'], #"RACE: Two or more races: Two races excluding Some other race, and three or more races: Total population -- (Estimate)"
    ['B02008e1','r_mix4'], #WHITE ALONE OR IN COMBINATION WITH ONE OR MORE OTHER RACES: Total: White alone or in combination with one or more other races -- (Estimate)
    ['B02009e1','r_mix5'], #BLACK OR AFRICAN AMERICAN ALONE OR IN COMBINATION WITH ONE OR MORE OTHER RACES: Total: Black or African American alone or in combination with one or more other races -- (Estimate)
    ['B02010e1','r_mix6'], #AMERICAN INDIAN AND ALASKA NATIVE ALONE OR IN COMBINATION WITH ONE OR MORE OTHER RACES: Total: People who are American Indian or Alaska Native alone or in combination with one or more other races -- (Estimate)
    ['B02011e1','r_mix7'], #ASIAN ALONE OR IN COMBINATION WITH ONE OR MORE OTHER RACES: Total: Asian alone or in combination with one or more other races -- (Estimate)
    ['B02012e1','r_mix8'], #NATIVE HAWAIIAN AND OTHER PACIFIC ISLANDER ALONE OR IN COMBINATION WITH ONE OR MORE OTHER RACES: Total: Native Hawaiian and Other Pacific Islander alone or in combination with one or more other races -- (Estimate)
    ['B02013e1','r_mix9'], #SOME OTHER RACE ALONE OR IN COMBINATION WITH ONE OR MORE OTHER RACES: Total: Some other race alone or in combination with one or more other races -- (Estimate)

    ['B15003e1','e_total'],  #EDUCATIONAL ATTAINMENT FOR THE POPULATION 25 YEARS AND OVER: Total: Population 25 years and over -- (Estimate)
    ['B15003e2','e_none'],  #EDUCATIONAL ATTAINMENT FOR THE POPULATION 25 YEARS AND OVER: No schooling completed: Population 25 years and over -- (Estimate)
    ['B15003e3','e_nursery'],  #EDUCATIONAL ATTAINMENT FOR THE POPULATION 25 YEARS AND OVER: Nursery school: Population 25 years and over -- (Estimate)
    ['B15003e4','e_k'],  #EDUCATIONAL ATTAINMENT FOR THE POPULATION 25 YEARS AND OVER: Kindergarten: Population 25 years and over -- (Estimate)
    ['B15003e5','e_1'],  #EDUCATIONAL ATTAINMENT FOR THE POPULATION 25 YEARS AND OVER: 1st grade: Population 25 years and over -- (Estimate)
    ['B15003e6','e_2'],  #EDUCATIONAL ATTAINMENT FOR THE POPULATION 25 YEARS AND OVER: 2nd grade: Population 25 years and over -- (Estimate)
    ['B15003e7','e_3'],  #EDUCATIONAL ATTAINMENT FOR THE POPULATION 25 YEARS AND OVER: 3rd grade: Population 25 years and over -- (Estimate)
    ['B15003e8','e_4'],  #EDUCATIONAL ATTAINMENT FOR THE POPULATION 25 YEARS AND OVER: 4th grade: Population 25 years and over -- (Estimate)
    ['B15003e9','e_5'],  #EDUCATIONAL ATTAINMENT FOR THE POPULATION 25 YEARS AND OVER: 5th grade: Population 25 years and over -- (Estimate)
    ['B15003e10','e_6'], #EDUCATIONAL ATTAINMENT FOR THE POPULATION 25 YEARS AND OVER: 6th grade: Population 25 years and over -- (Estimate)
    ['B15003e11','e_7'], #EDUCATIONAL ATTAINMENT FOR THE POPULATION 25 YEARS AND OVER: 7th grade: Population 25 years and over -- (Estimate)
    ['B15003e12','e_8'], #EDUCATIONAL ATTAINMENT FOR THE POPULATION 25 YEARS AND OVER: 8th grade: Population 25 years and over -- (Estimate)
    ['B15003e13','e_9'], #EDUCATIONAL ATTAINMENT FOR THE POPULATION 25 YEARS AND OVER: 9th grade: Population 25 years and over -- (Estimate)
    ['B15003e14','e_10'], #EDUCATIONAL ATTAINMENT FOR THE POPULATION 25 YEARS AND OVER: 10th grade: Population 25 years and over -- (Estimate)
    ['B15003e15','e_11'], #EDUCATIONAL ATTAINMENT FOR THE POPULATION 25 YEARS AND OVER: 11th grade: Population 25 years and over -- (Estimate)
    ['B15003e16','e_12_no_diploma'], #"EDUCATIONAL ATTAINMENT FOR THE POPULATION 25 YEARS AND OVER: 12th grade, no diploma: Population 25 years and over -- (Estimate)"
    ['B15003e17','e_hs'], #EDUCATIONAL ATTAINMENT FOR THE POPULATION 25 YEARS AND OVER: Regular high school diploma: Population 25 years and over -- (Estimate)
    ['B15003e18','e_ged'], #EDUCATIONAL ATTAINMENT FOR THE POPULATION 25 YEARS AND OVER: GED or alternative credential: Population 25 years and over -- (Estimate)
    ['B15003e19','e_col1'], #"EDUCATIONAL ATTAINMENT FOR THE POPULATION 25 YEARS AND OVER: Some college, less than 1 year: Population 25 years and over -- (Estimate)"
    ['B15003e20','e_col1_no_deg'], #"EDUCATIONAL ATTAINMENT FOR THE POPULATION 25 YEARS AND OVER: Some college, 1 or more years, no degree: Population 25 years and over -- (Estimate)"
    ['B15003e21','e_asso'], #EDUCATIONAL ATTAINMENT FOR THE POPULATION 25 YEARS AND OVER: Associate's degree: Population 25 years and over -- (Estimate)
    ['B15003e22','e_bach'], #EDUCATIONAL ATTAINMENT FOR THE POPULATION 25 YEARS AND OVER: Bachelor's degree: Population 25 years and over -- (Estimate)
    ['B15003e23','e_mast'], #EDUCATIONAL ATTAINMENT FOR THE POPULATION 25 YEARS AND OVER: Master's degree: Population 25 years and over -- (Estimate)
    ['B15003e24','e_prof'], #EDUCATIONAL ATTAINMENT FOR THE POPULATION 25 YEARS AND OVER: Professional school degree: Population 25 years and over -- (Estimate)
    ['B15003e25','e_doct'], #EDUCATIONAL ATTAINMENT FOR THE POPULATION 25 YEARS AND OVER: Doctorate degree: Population 25 years and over -- (Estimate)

    ['B19001e1','hi_total'],  #HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS): Total: Households -- (Estimate)
    ['B19001e2','hi_0_9'],  #"HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS): Less than $10,000: Households -- (Estimate)"
    ['B19001e3','hi_10_14'],  #"HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS): $10,000 to $14,999: Households -- (Estimate)"
    ['B19001e4','hi_15_19'],  #"HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS): $15,000 to $19,999: Households -- (Estimate)"
    ['B19001e5','hi_20_24'],  #"HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS): $20,000 to $24,999: Households -- (Estimate)"
    ['B19001e6','hi_25_29'],  #"HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS): $25,000 to $29,999: Households -- (Estimate)"
    ['B19001e7','hi_30_34'],  #"HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS): $30,000 to $34,999: Households -- (Estimate)"
    ['B19001e8','hi_35_39'],  #"HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS): $35,000 to $39,999: Households -- (Estimate)"
    ['B19001e9','hi_40_45'],  #"HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS): $40,000 to $44,999: Households -- (Estimate)"
    ['B19001e10','hi_45_49'], #"HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS): $45,000 to $49,999: Households -- (Estimate)"
    ['B19001e11','hi_50_59'], #"HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS): $50,000 to $59,999: Households -- (Estimate)"
    ['B19001e12','hi_60_74'], #"HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS): $60,000 to $74,999: Households -- (Estimate)"
    ['B19001e13','hi_75_99'], #"HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS): $75,000 to $99,999: Households -- (Estimate)"
    ['B19001e14','hi_100_124'], #"HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS): $100,000 to $124,999: Households -- (Estimate)"
    ['B19001e15','hi_125_149'], #"HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS): $125,000 to $149,999: Households -- (Estimate)"
    ['B19001e16','hi_150_199'], #"HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS): $150,000 to $199,999: Households -- (Estimate)"
    ['B19001e17','hi_200_plus'], #"HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS): $200,000 or more: Households -- (Estimate)"

    ['B19101e1','fi_total'], #FAMILY INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS): Total: Families -- (Estimate)
    ['B19101e2','fi_0_9'], #"FAMILY INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS): Less than $10,000: Families -- (Estimate)"
    ['B19101e3','fi_10_14'], #"FAMILY INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS): $10,000 to $14,999: Families -- (Estimate)"
    ['B19101e4','fi_15_19'], #"FAMILY INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS): $15,000 to $19,999: Families -- (Estimate)"
    ['B19101e5','fi_20_24'], #"FAMILY INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS): $20,000 to $24,999: Families -- (Estimate)"
    ['B19101e6','fi_25_29'], #"FAMILY INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS): $25,000 to $29,999: Families -- (Estimate)"
    ['B19101e7','fi_30_34'], #"FAMILY INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS): $30,000 to $34,999: Families -- (Estimate)"
    ['B19101e8','fi_35_39'], #"FAMILY INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS): $35,000 to $39,999: Families -- (Estimate)"
    ['B19101e9','fi_40_44'], #"FAMILY INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS): $40,000 to $44,999: Families -- (Estimate)"
    ['B19101e10','fi_45_49'], #"FAMILY INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS): $45,000 to $49,999: Families -- (Estimate)"
    ['B19101e11','fi_50_59'], #"FAMILY INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS): $50,000 to $59,999: Families -- (Estimate)"
    ['B19101e12','fi_60_74'], #"FAMILY INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS): $60,000 to $74,999: Families -- (Estimate)"
    ['B19101e13','fi_75_99'], #"FAMILY INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS): $75,000 to $99,999: Families -- (Estimate)"
    ['B19101e14','fi_100_124'], #"FAMILY INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS): $100,000 to $124,999: Families -- (Estimate)"
    ['B19101e15','fi_125_149'], #"FAMILY INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS): $125,000 to $149,999: Families -- (Estimate)"
    ['B19101e16','fi_150_199'], #"FAMILY INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS): $150,000 to $199,999: Families -- (Estimate)"
    ['B19101e17','fi_200_plus'], #"FAMILY INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS): $200,000 or more: Families -- (Estimate)"

    ['B19301e1','p_c_i'], #PER CAPITA INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS): Total: Total population -- (Estimate)

    ['C17002e1','pir_total'], #RATIO OF INCOME TO POVERTY LEVEL IN THE PAST 12 MONTHS: Total: Population for whom poverty status is determined -- (Estimate)
    ['C17002e2','pir_000_049'], #RATIO OF INCOME TO POVERTY LEVEL IN THE PAST 12 MONTHS: Under .50: Population for whom poverty status is determined -- (Estimate)
    ['C17002e3','pir_050_099'], #RATIO OF INCOME TO POVERTY LEVEL IN THE PAST 12 MONTHS: .50 to .99: Population for whom poverty status is determined -- (Estimate)
    ['C17002e4','pir_100_124'], #RATIO OF INCOME TO POVERTY LEVEL IN THE PAST 12 MONTHS: 1.00 to 1.24: Population for whom poverty status is determined -- (Estimate)
    ['C17002e5','pir_125_149'], #RATIO OF INCOME TO POVERTY LEVEL IN THE PAST 12 MONTHS: 1.25 to 1.49: Population for whom poverty status is determined -- (Estimate)
    ['C17002e6','pir_150_184'], #RATIO OF INCOME TO POVERTY LEVEL IN THE PAST 12 MONTHS: 1.50 to 1.84: Population for whom poverty status is determined -- (Estimate)
    ['C17002e7','pir_185_199'], #RATIO OF INCOME TO POVERTY LEVEL IN THE PAST 12 MONTHS: 1.85 to 1.99: Population for whom poverty status is determined -- (Estimate)
    ['C17002e8','pir_200_plus'], #RATIO OF INCOME TO POVERTY LEVEL IN THE PAST 12 MONTHS: 2.00 and over: Population for whom poverty status is determined -- (Estimate)

    ['B27010e1','unins_total'], #TYPES OF HEALTH INSURANCE COVERAGE BY AGE: Total: Civilian noninstitutionalized population -- (Estimate)
    ['B27010e17','unins_0_18'], #TYPES OF HEALTH INSURANCE COVERAGE BY AGE: Under 18 years: No health insurance coverage: Civilian noninstitutionalized population -- (Estimate)
    ['B27010e33','unins_18_34'], #TYPES OF HEALTH INSURANCE COVERAGE BY AGE: 18 to 34 years: No health insurance coverage: Civilian noninstitutionalized population -- (Estimate)
    ['B27010e50','unins_35_64'], #TYPES OF HEALTH INSURANCE COVERAGE BY AGE: 35 to 64 years: No health insurance coverage: Civilian noninstitutionalized population -- (Estimate)
    ['B27010e66','unins_65_plus'], #TYPES OF HEALTH INSURANCE COVERAGE BY AGE: 65 years and over: No health insurance coverage: Civilian noninstitutionalized population -- (Estimate)

]

# Build Census Data with Selected Columns

In [77]:
field_ids = []
for field in fields:
    field_ids.append(field[0])
field_ids.insert(0, "census_block_group")

In [78]:
# Select only columns needed and drop duplicates
census_data = census_data[field_ids]
census_data = census_data.loc[:,~census_data.columns.duplicated()]

In [79]:
# Add FIPS code
census_data['fips_code'] = census_data['census_block_group'].astype('str').str[:5]

In [80]:
# Group by the new fips code, and sum the values
del field_ids[0]
census_data = census_data.groupby(['fips_code'])[field_ids].sum().reset_index()

In [81]:
# Rename all the columns to the human readable versions
for field in fields:
    census_data = census_data.rename(columns={field[0]:field[1]})

In [82]:
census_data.head()

,fips_code,pop_total,m_total,m_0_5,m_5_9,m_10_14,m_15_17,m_18_19,m_20_20,m_21_21,...,pir_100_124,pir_125_149,pir_150_184,pir_185_199,pir_200_plus,unins_total,unins_0_18,unins_18_34,unins_35_64,unins_65_plus
0,10001,171474,82687,5678,5534,5951,3443,2578,1766,1402,...,8497,7747,12205,4637,111052,168146,2032,5260,5498,270
1,10003,551997,267276,16832,16455,17992,10852,8324,3766,4164,...,16301,19718,28021,10523,399898,543604,4448,15617,16662,971
2,10005,211224,102453,5834,5870,5872,3381,2010,1239,1136,...,8717,9700,13434,5637,143438,208518,1806,6699,9252,262
3,10010,55049,26877,1680,2003,2112,1327,727,442,464,...,2452,3467,3552,1616,36814,54387,350,1968,2527,7
4,10030,199510,97371,5876,6197,6399,4106,2208,947,1073,...,9842,8757,12551,7111,132828,197046,1797,9639,11611,208


In [83]:
census_data.shape

(3281, 139)

In [84]:
census_data[census_data.fips_code.str.contains("6001")]

,fips_code,pop_total,m_total,m_0_5,m_5_9,m_10_14,m_15_17,m_18_19,m_20_20,m_21_21,...,pir_100_124,pir_125_149,pir_150_184,pir_185_199,pir_200_plus,unins_total,unins_0_18,unins_18_34,unins_35_64,unins_65_plus
306,16001,425798,213344,13428,15320,16478,9309,5187,3173,2871,...,14589,18204,27374,11679,294983,419643,6183,17909,20109,514
1016,26001,10461,5277,154,219,193,145,85,32,50,...,623,583,1211,277,6058,10361,41,303,572,0
1613,36001,307891,148964,7921,7951,8432,5523,6297,3015,3727,...,10564,9137,13490,5334,215812,304048,1060,8307,6793,163
2994,56001,37836,19767,1080,766,994,492,1249,1037,1082,...,2056,1484,1978,383,20580,37625,235,1867,1149,37
3017,60014,1604534,786376,49855,50533,47230,28148,20039,9921,10205,...,57150,55622,76913,32021,1165679,1593510,11876,57313,61629,2397
3018,60019,683,354,10,0,2,6,0,0,7,...,0,8,14,3,618,683,0,7,12,0


# FIPS Metadata

In [85]:
data_fips.state_fips = data_fips.state_fips.astype(str).apply(lambda x: x.zfill(2))

In [86]:
data_fips.county_fips = data_fips.county_fips.astype(str).apply(lambda x: x.zfill(3))

In [87]:
data_fips['fips'] = data_fips['state_fips'] + data_fips['county_fips']

In [88]:
data_fips[data_fips.state=='CA'].head()

,state,state_fips,county_fips,county,class_code,fips
186,CA,06,001,Alameda County,H1,06001
187,CA,06,003,Alpine County,H1,06003
188,CA,06,005,Amador County,H1,06005
189,CA,06,007,Butte County,H1,06007
190,CA,06,009,Calaveras County,H1,06009


# Geographic Metadata

In [89]:
data_geo.census_block_group = data_geo.census_block_group.astype(str).apply(lambda x: x.zfill(12))

In [90]:
data_geo['fips'] = data_geo['census_block_group'].str[:5]

In [91]:
data_land = data_geo.groupby(['fips'])[['amount_land', 'amount_water']].sum()

In [92]:
data_land.amount_land = data_land.amount_land.astype('int64')

In [93]:
data_land.reset_index(level=0, inplace=True)

# Ouput to Pickle

In [33]:
with open('safegraph_census.p', 'wb') as f:
    pickle.dump(census_data, f)

In [34]:
with open('safegraph_ref_data_fips.p', 'wb') as f:
    pickle.dump(data_fips, f)

In [35]:
with open('safegraph_land.p', 'wb') as f:
    pickle.dump(data_land, f)